# ffm_mxnetVersion0.1

## 首先测试mxnet是否具有所需要的能力

### 关于每次只使用部分的参数

In [1]:
import mxnet as mx
from mxnet import ndarray as nd
import mxnet.autograd as ag

In [2]:
weight = nd.array([1,2,3,4,5])

weight.attach_grad()

In [3]:

with ag.record():
    loss = nd.array([1,2,3])*weight[:3]

loss.backward()
print(weight.grad)


[1. 2. 3. 0. 0.]
<NDArray 5 @cpu(0)>


### 关于能够通过抽取的方式取出参数更新

In [7]:
with ag.record():
    tempdata  = weight[[3,4,3]]
    tempdata2 = weight[[2,1,3]]
    loss = nd.array([1,2,3])*tempdata*tempdata2
    
loss.backward()
print(weight.grad)


[ 0. 10.  4. 27.  4.]
<NDArray 5 @cpu(0)>


In [8]:
import numpy as np
temp = np.array([1,2,3])
temp[1:2]

array([2])

### 关于三维数组的使用

In [9]:
nd.random_normal(shape=(2,2))


[[1.1630785  0.4838046 ]
 [0.29956347 0.15302546]]
<NDArray 2x2 @cpu(0)>

## 其次完成对于矩阵按照index进行存取的方法

In [10]:
index_v1 = ['a','b','c']
index_v2 = ['0','1','2','3','4','5']
array = nd.random_normal(shape = (3,5))

In [11]:
# 我希望做这样的一个东西：
# array[['a'.'a'],['2','2','4']]

#### 解决方法：operator.itemgetter

####  step 1:  build a dict to get a map from name to index

In [12]:
rowIndex = {'a':0,'b':1,'c':2}
colIndex = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5}

#### step 2: transfor the name to real index

In [13]:
from operator import itemgetter
rowneed = ['a','a']
colneed = ['2','2','4']
rowReal = itemgetter(*rowneed)(rowIndex)
colReal = itemgetter(*colneed)(colIndex)

In [14]:
print(rowReal,colReal)

(0, 0) (2, 2, 4)


### step 3:  get the value from dict , and change it 

In [15]:
array = nd.random_normal(shape = (3,5))

In [16]:
#use these mathod to get special row & cols
def selectM(list1, list2):
    row = np.array([[i] * len(list2) for i in list1])
    col = np.array(list2 * len(list1)).reshape(len(list1), len(list2))
    return row,col

In [17]:
row,col = selectM(rowReal,colReal)
print(row)
print(col)


[[0 0 0]
 [0 0 0]]
[[2 2 4]
 [2 2 4]]


In [18]:
array[row,col] = 1

In [19]:
array


[[-0.6141325   1.8307649   1.          0.05383794  1.        ]
 [-0.5916499   0.85860497 -0.22794183  0.20131476  0.3500547 ]
 [ 0.5360521   1.5194443   1.9040879  -1.5734432  -0.14007866]]
<NDArray 3x5 @cpu(0)>

In [20]:
np.ones((4000,5)).dot(np.ones((5,))).shape

(4000,)

### make a function to use later

In [21]:
import numpy as np
rowIndex = {'a':0,'b':1,'c':2}
colIndex = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5}
rowneed = ['a','a']
colneed = ['2','2','4']
array = nd.random_normal(shape = (3,5))



def selectM(list1, list2):
    row = np.array([[i] * len(list2) for i in list1])
    col = np.array(list2 * len(list1)).reshape(len(list1), len(list2))
    return row,col

def askByIndex(rowneed,colneed,rowIndex,colIndex):
    from operator import itemgetter
    rowReal = itemgetter(*rowneed)(rowIndex)
    colReal = itemgetter(*colneed)(colIndex)
    row,col = selectM(rowReal,colReal)
    return row,col

row,col = askByIndex(rowneed,colneed,rowIndex,colIndex)
array[row,col]


[[ 0.5035904   0.5035904  -0.55021375]
 [ 0.5035904   0.5035904  -0.55021375]]
<NDArray 2x3 @cpu(0)>

### make it more easier

In [22]:
# make it more easier:

class DataManageMent(object):
    def __init__(self,rowIndex,colIndex):
        self._rowIndex = rowIndex
        self._colIndex = colIndex
    
    def selectM(self,list1, list2):
        row = np.array([[i] * len(list2) for i in list1])
        col = np.array(list2 * len(list1)).reshape(len(list1), len(list2))
        return row,col

    def askByIndex(self,rowneed,colneed):
        from operator import itemgetter
        rowReal = itemgetter(*rowneed)(self._rowIndex)
        colReal = itemgetter(*colneed)(self._colIndex)
        row,col = selectM(rowReal,colReal)
        return row,col
    def getByIndex(self,array,rowneed,colneed):
        row , col = self.askByIndex(rowneed,colneed)
        return array[row,col]


In [23]:
import numpy as np

# build a datamanager:
rowIndex = {'a':0,'b':1,'c':2}
colIndex = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5}
datamanager  = DataManageMent(rowIndex,colIndex)


rowneed = ['a','a']
colneed = ['2','2','4']
array = nd.random_normal(shape = (3,5))


datamanager.getByIndex(array,rowneed,colneed)



[[ 1.1119636  1.1119636 -1.1739492]
 [ 1.1119636  1.1119636 -1.1739492]]
<NDArray 2x3 @cpu(0)>

# mxnet version

In [4]:
import pandas as pd
import numpy as np
import os
np.set_printoptions(precision=2,linewidth=100)

In [5]:
trainSet = pd.read_csv("train.tiny-Copy1.csv")
trainSet.head()


,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000000,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,10000001,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,10000002,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,10000003,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,10000004,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


### 暂时跳过缺失值处理

In [6]:
trainSet = trainSet.dropna(how = 'any')
trainSet.head(4)

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
35,10000035,1,0.0,127,1.0,3.0,1683.0,19.0,26.0,17.0,...,8efede7f,bd17c3da,f6a3e43b,a458ea53,35cd95c9,ad3062eb,c7dc6720,3fdb382b,010f6491,49d68486
256,10000256,0,4.0,0,128.0,42.0,7.0,44.0,4.0,44.0,...,27c07bd6,bc5a0ff7,083e89d9,a458ea53,ac3aac58,c9d4222a,32c7478e,8e183052,001f3601,b4aa4b3d
328,10000328,0,3.0,0,27.0,26.0,25.0,37.0,14.0,22.0,...,e5ba7672,456d734d,4764bf77,a458ea53,52fa6318,ad3062eb,3a171ecb,3e90e1b2,724b04da,26c64313
338,10000338,0,0.0,1,2.0,14.0,2109.0,43.0,7.0,18.0,...,3486227d,df4fffb7,21ddcdc9,b1252a9d,53fbce39,c9d4222a,32c7478e,57ef7a21,010f6491,779ff446


### 归一处理
- 注意这里I3的值很大，在处理过程中会出事的，所以必要时需要对于连续变量做归一化处理

In [7]:
pass

### 取出局部资源来调试算法
- 注意：实际中应当把训练集和测试集统一进行编号

In [8]:
trainSet = trainSet.loc[:,['Label','I1','I2','C17','C18','C19']].head()
trainSet.reset_index(inplace=True,drop=True)
trainSet

,Label,I1,I2,C17,C18,C19
0,1,0.0,127,8efede7f,bd17c3da,f6a3e43b
1,0,4.0,0,27c07bd6,bc5a0ff7,083e89d9
2,0,3.0,0,e5ba7672,456d734d,4764bf77
3,0,0.0,1,3486227d,df4fffb7,21ddcdc9
4,0,1.0,0,27c07bd6,a78bd508,21ddcdc9


### change type so it can be done

In [9]:
target = trainSet.Label
trainSet = trainSet.drop( 'Label',axis=1)
categories = ['C17','C18','C19']
continues  = ['I1','I2']

# attentsion : 这样的apply是一个逐列的处理
trainSet.loc[:,categories] = trainSet.loc[:,categories].apply(lambda x : x.astype('category'))
trainSet.loc[:,continues] = trainSet.loc[:,continues].apply(lambda x : x.astype('float32'))


### get dummies

In [10]:
xdata_trnsf = pd.get_dummies(trainSet,prefix=categories,prefix_sep = ":",sparse = True)


In [11]:
# xdata_trnsf = pd.get_dummies(trainSet,prefix=categories,prefix_sep = ":",sparse = True)
xdata_trnsf

,I1,I2,C17:27c07bd6,C17:3486227d,C17:8efede7f,C17:e5ba7672,C18:456d734d,C18:a78bd508,C18:bc5a0ff7,C18:bd17c3da,C18:df4fffb7,C19:083e89d9,C19:21ddcdc9,C19:4764bf77,C19:f6a3e43b
0,0.0,127.0,0,0,1,0,0,0,0,1,0,0,0,0,1
1,4.0,0.0,1,0,0,0,0,0,1,0,0,1,0,0,0
2,3.0,0.0,0,0,0,1,1,0,0,0,0,0,0,1,0
3,0.0,1.0,0,1,0,0,0,0,0,0,1,0,1,0,0
4,1.0,0.0,1,0,0,0,0,1,0,0,0,0,1,0,0


In [12]:
# 拿到一级label和二级label,重构索引
import re
label_v1 = [ label.split(':')[0] for label in xdata_trnsf.columns.tolist()]
label_v2 = xdata_trnsf.columns.tolist()
index = pd.MultiIndex.from_arrays([label_v1,label_v2],names = ['field_id','index_id'])
xdata_trnsf.columns = index
xdata_trnsf.head()

field_id   I1     I2          C17                                         \
index_id   I1     I2 C17:27c07bd6 C17:3486227d C17:8efede7f C17:e5ba7672   
0         0.0  127.0            0            0            1            0   
1         4.0    0.0            1            0            0            0   
2         3.0    0.0            0            0            0            1   
3         0.0    1.0            0            1            0            0   
4         1.0    0.0            1            0            0            0   

field_id          C18                                                      \
index_id C18:456d734d C18:a78bd508 C18:bc5a0ff7 C18:bd17c3da C18:df4fffb7   
0                   0            0            0            1            0   
1                   0            0            1            0            0   
2                   1            0            0            0            0   
3                   0            0            0            0            1   
4                   0            1            0            0            0   

field_id          C19                                         
index_id C19:083e89d9 C19:21ddcdc9 C19:4764bf77 C19:f6a3e43b  
0                   0            0            0            1  
1                   1            0            0            0  
2                   0            0            1            0  
3                   0            1            0            0  
4                   0            1            0            0

### 制作字典，存储从名字到index 的映射

In [13]:
# define a function to print dict:
def printDict(d):
    import json
    print(json.dumps(d, indent = 4))

In [14]:
def createIndex(label):
    from collections import OrderedDict
    from collections import Counter
    #print(label)
    orderCol = list(OrderedDict.fromkeys(label))
    #print(orderCol)
    label_dict = dict(zip(orderCol,range(len(orderCol))))
    return label_dict

In [15]:
# encode:
label_v1_dict = createIndex(label_v1)
printDict(label_v1_dict)
label_v2_dict = createIndex(label_v2)
printDict(label_v2_dict)

{
    "I1": 0,
    "I2": 1,
    "C17": 2,
    "C18": 3,
    "C19": 4
}
{
    "I1": 0,
    "I2": 1,
    "C17:27c07bd6": 2,
    "C17:3486227d": 3,
    "C17:8efede7f": 4,
    "C17:e5ba7672": 5,
    "C18:456d734d": 6,
    "C18:a78bd508": 7,
    "C18:bc5a0ff7": 8,
    "C18:bd17c3da": 9,
    "C18:df4fffb7": 10,
    "C19:083e89d9": 11,
    "C19:21ddcdc9": 12,
    "C19:4764bf77": 13,
    "C19:f6a3e43b": 14
}


### 制作字典， 存储从converted index到conterted filed的映射信息

In [16]:
from operator import itemgetter
indexToFiled = dict(zip(itemgetter(*label_v2)(label_v2_dict),
                itemgetter(*label_v1)(label_v1_dict)))
printDict(indexToFiled)

{
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 2,
    "4": 2,
    "5": 2,
    "6": 3,
    "7": 3,
    "8": 3,
    "9": 3,
    "10": 3,
    "11": 4,
    "12": 4,
    "13": 4,
    "14": 4
}


### 初始化模型参数

In [17]:
# tunning parameters
numOfField = label_v1_dict.__len__()
numOfIndex = label_v2_dict.__len__()
numOfK     = 1

In [18]:
import mxnet.ndarray as nd
import mxnet.autograd as ag
Wmatrix = nd.random_uniform(shape=(numOfK,numOfField,numOfIndex))
Wmatrix.attach_grad()
print(Wmatrix)
print(Wmatrix.grad)


[[[5.49e-01 5.93e-01 7.15e-01 8.44e-01 6.03e-01 8.58e-01 5.45e-01 8.47e-01 4.24e-01 6.24e-01
   6.46e-01 3.84e-01 4.38e-01 2.98e-01 8.92e-01]
  [5.67e-02 9.64e-01 2.73e-01 3.83e-01 4.78e-01 7.92e-01 8.12e-01 5.29e-01 4.80e-01 5.68e-01
   3.93e-01 9.26e-01 8.36e-01 7.10e-02 3.37e-01]
  [8.71e-02 6.48e-01 2.02e-02 3.68e-01 8.33e-01 9.57e-01 7.78e-01 1.40e-01 4.17e-01 9.97e-01
   7.20e-01 9.33e-01 1.14e-04 1.28e-01 3.02e-01]
  [9.99e-01 1.47e-01 2.36e-01 9.23e-02 3.97e-01 1.86e-01 3.88e-01 3.46e-01 6.70e-01 3.97e-01
   9.36e-01 5.39e-01 8.46e-01 4.19e-01 3.13e-01]
  [6.85e-01 5.25e-01 2.04e-01 4.43e-01 8.78e-01 2.30e-01 2.74e-02 5.34e-01 6.70e-01 9.14e-01
   4.17e-01 4.57e-01 5.59e-01 4.31e-01 1.40e-01]]]
<NDArray 1x5x15 @cpu(0)>

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]
<NDArray 1x5x1

### SGD

#### 获取第一个人的信息

In [37]:
from IPython.display import display_html
from scipy.sparse import  coo_matrix,csr_matrix,lil_matrix
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

userNum = 0
record = xdata_trnsf.loc[3,:].to_frame().to_sparse()
record

3
field_id index_id         
I1       I1            0.0
I2       I2            1.0
C17      C17:27c07bd6  0.0
         C17:3486227d  1.0
         C17:8efede7f  0.0
         C17:e5ba7672  0.0
C18      C18:456d734d  0.0
         C18:a78bd508  0.0
         C18:bc5a0ff7  0.0
         C18:bd17c3da  0.0
         C18:df4fffb7  1.0
C19      C19:083e89d9  0.0
         C19:21ddcdc9  1.0
         C19:4764bf77  0.0
         C19:f6a3e43b  0.0


#### 生成交互矩阵

In [38]:
# 先看看交互矩阵长什么样
record = xdata_trnsf.loc[userNum,:].to_frame().to_sparse()
record
record_matrix = record.dot(record.T)
upper = np.triu(record_matrix.values)
upper[range(upper.shape[0]),range(upper.shape[0]) ]= 0
record_matrix = pd.DataFrame(upper  ,index = record_matrix.index , columns = record_matrix.columns)
record_matrix

field_id                I1   I2          C17                            \
index_id                I1   I2 C17:27c07bd6 C17:3486227d C17:8efede7f   
field_id index_id                                                        
I1       I1            0.0  0.0          0.0          0.0          0.0   
I2       I2            0.0  0.0          0.0          0.0        127.0   
C17      C17:27c07bd6  0.0  0.0          0.0          0.0          0.0   
         C17:3486227d  0.0  0.0          0.0          0.0          0.0   
         C17:8efede7f  0.0  0.0          0.0          0.0          0.0   
         C17:e5ba7672  0.0  0.0          0.0          0.0          0.0   
C18      C18:456d734d  0.0  0.0          0.0          0.0          0.0   
         C18:a78bd508  0.0  0.0          0.0          0.0          0.0   
         C18:bc5a0ff7  0.0  0.0          0.0          0.0          0.0   
         C18:bd17c3da  0.0  0.0          0.0          0.0          0.0   
         C18:df4fffb7  0.0  0.0          0.0          0.0          0.0   
C19      C19:083e89d9  0.0  0.0          0.0          0.0          0.0   
         C19:21ddcdc9  0.0  0.0          0.0          0.0          0.0   
         C19:4764bf77  0.0  0.0          0.0          0.0          0.0   
         C19:f6a3e43b  0.0  0.0          0.0          0.0          0.0   

field_id                                    C18                            \
index_id              C17:e5ba7672 C18:456d734d C18:a78bd508 C18:bc5a0ff7   
field_id index_id                                                           
I1       I1                    0.0          0.0          0.0          0.0   
I2       I2                    0.0          0.0          0.0          0.0   
C17      C17:27c07bd6          0.0          0.0          0.0          0.0   
         C17:3486227d          0.0          0.0          0.0          0.0   
         C17:8efede7f          0.0          0.0          0.0          0.0   
         C17:e5ba7672          0.0          0.0          0.0          0.0   
C18      C18:456d734d          0.0          0.0          0.0          0.0   
         C18:a78bd508          0.0          0.0          0.0          0.0   
         C18:bc5a0ff7          0.0          0.0          0.0          0.0   
         C18:bd17c3da          0.0          0.0          0.0          0.0   
         C18:df4fffb7          0.0          0.0          0.0          0.0   
C19      C19:083e89d9          0.0          0.0          0.0          0.0   
         C19:21ddcdc9          0.0          0.0          0.0          0.0   
         C19:4764bf77          0.0          0.0          0.0          0.0   
         C19:f6a3e43b          0.0          0.0          0.0          0.0   

field_id                                                 C19               \
index_id              C18:bd17c3da C18:df4fffb7 C19:083e89d9 C19:21ddcdc9   
field_id index_id                                                           
I1       I1                    0.0          0.0          0.0          0.0   
I2       I2                  127.0          0.0          0.0          0.0   
C17      C17:27c07bd6          0.0          0.0          0.0          0.0   
         C17:3486227d          0.0          0.0          0.0          0.0   
         C17:8efede7f          1.0          0.0          0.0          0.0   
         C17:e5ba7672          0.0          0.0          0.0          0.0   
C18      C18:456d734d          0.0          0.0          0.0          0.0   
         C18:a78bd508          0.0          0.0          0.0          0.0   
         C18:bc5a0ff7          0.0          0.0          0.0          0.0   
         C18:bd17c3da          0.0          0.0          0.0          0.0   
         C18:df4fffb7          0.0          0.0          0.0          0.0   
C19      C19:083e89d9          0.0          0.0          0.0          0.0   
         C19:21ddcdc9          0.0          0.0          0.0          0.0   
         C19:4764bf77          0.0          0.0 

In [39]:
# 生成交互矩阵
record = xdata_trnsf.iloc[userNum,:].to_frame().to_sparse()
record = csr_matrix(record.values)
record.toarray()

from scipy.sparse import triu
record_matrix = record.dot(record.T)
record_matrix = triu(record_matrix,format='lil')
record_diag = range(record_matrix.shape[0])
record_matrix[record_diag,record_diag ]= 0
record_matrix = record_matrix.tocoo()

In [40]:
print(record_matrix.toarray())

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 127.   0.   0.   0.   0. 127.   0.   0.   0.   0. 127.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   

In [41]:
# get weight from Wmatrix
# get original index
print(label_v1)
print(label_v2)

['I1', 'I2', 'C17', 'C17', 'C17', 'C17', 'C18', 'C18', 'C18', 'C18', 'C18', 'C19', 'C19', 'C19', 'C19']
['I1', 'I2', 'C17:27c07bd6', 'C17:3486227d', 'C17:8efede7f', 'C17:e5ba7672', 'C18:456d734d', 'C18:a78bd508', 'C18:bc5a0ff7', 'C18:bd17c3da', 'C18:df4fffb7', 'C19:083e89d9', 'C19:21ddcdc9', 'C19:4764bf77', 'C19:f6a3e43b']


In [42]:
# get row-col-value info from record_matrix
from operator import itemgetter
data = record_matrix.data
row  = record_matrix.row
col  = record_matrix.col
rowindex = itemgetter(*row)(indexToFiled)
colindex = itemgetter(*col)(indexToFiled)


### 数据展示

In [43]:

temp = pd.DataFrame([data,row,col,colindex,rowindex]).T
temp.columns = ["data","row","col","colindex","rowindex"]
temp.head()

,data,row,col,colindex,rowindex
0,127.0,1.0,4.0,2.0,1.0
1,127.0,1.0,9.0,3.0,1.0
2,127.0,1.0,14.0,4.0,1.0
3,1.0,4.0,9.0,3.0,2.0
4,1.0,4.0,14.0,4.0,2.0


### 计算$\phi$

In [44]:
### 从Wmtrix中调用涉及到的w值
phi = (Wmatrix[:,rowindex,col] * Wmatrix[:,colindex,row] * nd.array(data)).sum()
phi


[73.33]
<NDArray 1 @cpu(0)>

#### test auto grad

$$ loss = \min_{\mathbf{w}} \sum_{i=1}^L \log \big( 1 + \exp\{ -y_i \phi (\mathbf{w}, \mathbf{x}_i ) \} \big) + \frac{\lambda}{2} \| \mathbf{w} \|^2$$

In [46]:
np.set_printoptions(precision=2,linewidth=200)
lamda = 0.001
y = target.loc[userNum]
with ag.record():
    phi = (Wmatrix[:,rowindex,col] * Wmatrix[:,colindex,row] * nd.array(data)).sum()
    loss = nd.log(1+nd.exp(-y*phi)) + lamda*(Wmatrix*Wmatrix).sum()/2
print(loss)
loss.backward()
Wmatrix.grad


[0.01]
<NDArray 1 @cpu(0)>



[[[5.49e-04 5.93e-04 7.15e-04 8.44e-04 6.03e-04 8.58e-04 5.45e-04 8.47e-04 4.24e-04 6.24e-04 6.46e-04 3.84e-04 4.38e-04 2.98e-04 8.92e-04]
  [5.67e-05 9.64e-04 2.73e-04 3.83e-04 4.78e-04 7.92e-04 8.12e-04 5.29e-04 4.80e-04 5.68e-04 3.93e-04 9.26e-04 8.36e-04 7.10e-05 3.37e-04]
  [8.71e-05 6.48e-04 2.02e-05 3.68e-04 8.33e-04 9.57e-04 7.78e-04 1.40e-04 4.17e-04 9.97e-04 7.20e-04 9.33e-04 1.14e-07 1.28e-04 3.02e-04]
  [9.99e-04 1.47e-04 2.36e-04 9.23e-05 3.97e-04 1.86e-04 3.88e-04 3.46e-04 6.70e-04 3.97e-04 9.36e-04 5.39e-04 8.46e-04 4.19e-04 3.13e-04]
  [6.85e-04 5.25e-04 2.04e-04 4.43e-04 8.78e-04 2.30e-04 2.74e-05 5.34e-04 6.70e-04 9.14e-04 4.17e-04 4.57e-04 5.59e-04 4.31e-04 1.40e-04]]]
<NDArray 1x5x15 @cpu(0)>

In [47]:
y

1

### SGD主流程

In [61]:
np.set_printoptions(precision=4,linewidth=100)
# 生成交互矩阵
# get three row each time
from scipy.sparse import triu
from scipy.sparse import vstack
from operator import itemgetter
from  numpy.random import shuffle


def getPhi(userNum):
    record = xdata_trnsf.iloc[userNum,:].to_frame().to_sparse()
    record = csr_matrix(record.values)
    record_matrix = record.dot(record.T)
    record_matrix = triu(record_matrix,format='lil')
    record_diag = range(record_matrix.shape[0])
    record_matrix[record_diag,record_diag ]= 0
    record_matrix = record_matrix.tocoo()

    data = record_matrix.data
    row  = record_matrix.row
    col  = record_matrix.col
    rowindex = itemgetter(*row)(indexToFiled)
    colindex = itemgetter(*col)(indexToFiled)
    phi = (Wmatrix[:,rowindex,col] * Wmatrix[:,colindex,row] * nd.array(data)).sum()
    return phi




learning_rate = 0.001
lamda         = 0.001

userList = list(range(xdata_trnsf.shape[0]))
shuffle(userList)

for userNum in userList:
    with ag.record():
        phi = getPhi(userNum)
        loss = nd.log(1+nd.exp(-target.loc[userNum]*phi)) + lamda*(Wmatrix*Wmatrix).sum()/2
    
    loss.backward()
    #Wmatrix = Wmatrix - learning_rate*Wmatrix.grad

    
# from scipy.sparse import triu
# record_matrix = record.dot(record.T)
# record_matrix = triu(record_matrix,format='lil')
# record_diag = range(record_matrix.shape[0])
# record_matrix[record_diag,record_diag ]= 0
# record_matrix = record_matrix.tocoo()

MXNetError: [00:02:40] C:\projects\mxnet-distro-win\mxnet-build\src\imperative\imperative.cc:372: Check failed: !AGInfo::IsNone(*i) Cannot differentiate node because it is not in a computational graph. You need to set is_recording to true or use autograd.record() to save computational graphs for backward. If you want to differentiate the same graph twice, you need to pass retain_graph=True to backward.

In [67]:
userNum = 0
with ag.record():
    phi = getPhi(userNum)
    loss = nd.log(1+nd.exp(-target.loc[userNum]*phi)) + lamda*(Wmatrix*Wmatrix).sum()/2




[0.0128]
<NDArray 1 @cpu(0)>


MXNetError: [00:03:59] C:\projects\mxnet-distro-win\mxnet-build\src\imperative\imperative.cc:372: Check failed: !AGInfo::IsNone(*i) Cannot differentiate node because it is not in a computational graph. You need to set is_recording to true or use autograd.record() to save computational graphs for backward. If you want to differentiate the same graph twice, you need to pass retain_graph=True to backward.

In [64]:
Wmatrix


[[[5.4881e-01 5.9284e-01 7.1519e-01 8.4426e-01 6.0276e-01 8.5794e-01 5.4488e-01 8.4725e-01
   4.2365e-01 6.2356e-01 6.4589e-01 3.8438e-01 4.3759e-01 2.9753e-01 8.9177e-01]
  [5.6713e-02 9.6366e-01 2.7266e-01 3.8344e-01 4.7766e-01 7.9172e-01 8.1217e-01 5.2889e-01
   4.7998e-01 5.6804e-01 3.9278e-01 9.2560e-01 8.3608e-01 7.1036e-02 3.3740e-01]
  [8.7129e-02 6.4817e-01 2.0218e-02 3.6824e-01 8.3262e-01 9.5715e-01 7.7816e-01 1.4035e-01
   4.1702e-01 9.9718e-01 7.2032e-01 9.3256e-01 1.1438e-04 1.2812e-01 3.0233e-01]
  [9.9904e-01 1.4676e-01 2.3609e-01 9.2339e-02 3.9658e-01 1.8626e-01 3.8791e-01 3.4556e-01
   6.6975e-01 3.9677e-01 9.3554e-01 5.3882e-01 8.4631e-01 4.1919e-01 3.1327e-01]
  [6.8522e-01 5.2455e-01 2.0445e-01 4.4345e-01 8.7812e-01 2.2958e-01 2.7388e-02 5.3441e-01
   6.7047e-01 9.1396e-01 4.1730e-01 4.5720e-01 5.5869e-01 4.3070e-01 1.4039e-01]]]
<NDArray 1x5x15 @cpu(0)>